# Requirements to run

We recommend generatating a new Anaconda environment before trying to run this project.
The reason is that igraph may have dependencies that conflict with your other project configurations.
As a consequence a new Anaconda environment will likely lead to less debugging and conflicting package issues.

### If you miss a dependency
conda install -c conda-forge python-igraph <br/>
conda install -c conda-forge pycairo <br/>
conda install pandas

Note that the correct igraph package is listed at https://igraph.org/python/doc/tutorial/install.html and NOT at anaconda.org!

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import igraph as ig
import math
import numpy as np
import os as os

## Currently problematic UIs
10594 reason: mapview where determining observe time is challenging and order of interaction

## Deleted UIs
1996: reason: faulty json missing component for "Home" <br/>
21186 reason: Too many components with clickable flag set incorrectly. <br/>
24701 reason: incorrect clickability labeling <br/>
26150 reason: no clickables set and UI too complicated to reliably mark correct clickables <br/>
26442 reason: too much speculation when determining swipe component <br/>
29058 reason: no proper indicator component for swipeability <br/>

<br/>
11049 reason: charmaps to undefined, faulty coding (font not understood) <br/>
11148 reason: charmaps to undefined, faulty coding (font not understood) <br/>
11150 reason: charmaps to undefined, faulty coding (font not understood)

## Edited JSON files
14310 reason: the text items here are clickable in reality, original version had false <br/>
21545 reason: components not set as  clickable correctly <br/>
23689 reason: pager indicator means there is a swipe component but rico didn't register it as clickable
27046 reason: incorrectly set drawer as clickable when list items are the clickables <br/>
30348 reason: incorrectly set drawer as clickable when list items are the clickables <br/>
33395 reason: clickable text and arrow icons were incorrectly marked to false

# Notes on JSONs
33395: complexity outcome is more complicated due to text-icon split than with list item clickables due to how the interface has been defined. In reality there should be no complexity difference for the user. No action taken as of now. For this issue <br/>

### Component mappings
|Component | Code | Operation |
--- | --- | ---|
|Background Image| BI | Tap |
|Button Bar| BB| Tap|
|Checkbox | C | Tap |
|Card | CC | Tap |
|Icon | I| Tap |
|Input | IP | Input |
|Map View | MV | Multi |
| | | |


In [10]:
#give filename as "id.json"
#1809.json
#546.json

verticesMap = {"Advertisement": "AD",
               "Background Image": "BI",
               "Button Bar": "BB", 
               "Checkbox": "C", 
               "Card": "CC", 
               "Icon": "I", 
               "Image": "IC", 
               "Input": "IP", 
               "List Item": "L", 
               "Map View": "M",
               "Pager Indicator": "P",
               "Radio Button": "RB", 
               "Text": "TC", 
               "Text Button": "TB", 
               "Web View": "WV"}

edgesMap = {"AD": "Tap",
            "BB": "Tap",
            "BI": "Tap",
            "C": "Tap", 
            "CC": "Tap", 
            "I": "Tap", 
            "IC": "Tap", 
            "IP": "Input", 
            "L": "Tap", 
            "M": "Multi",
            "P": "Swipe",
            "RB": "Tap", 
            "TB": "Tap", 
            "TC": "Tap", 
            "WV": "Tap"} 

## Extract components from file

In [3]:
def getNormalizedBounds(dimensions, bounds):
    x1 = bounds[0] / dimensions[2]
    y1 = bounds[1] / dimensions[3]
    x2 = bounds[2] / dimensions[2]
    y2 = bounds[3] / dimensions[3]
    return [x1, y1, x2, y2]

def sortControlComponents(components):
    sortedComponents = []
    for component in components:
        bounds = component[1]
        componentPosition = 0
        for i in range(len(sortedComponents)):
            compareTo = sortedComponents[i]
            compareToBounds = compareTo[1]
            #If the new component is below the previous one then we move 
            #it to towards the end of component ordering
            if bounds[1] > compareToBounds[3]:
                componentPosition += 1
            #if we have overlap between the components we choose by
            #the component ordering in horizontal plane
            elif bounds[0] > compareToBounds[2]:
                componentPosition += 1
            #otherwise we assume that the current component is before the other component
        sortedComponents.insert(componentPosition, component)
    return sortedComponents
        

def getControlComponents(children):
    controlComponents = []
    def loopForControlComponents(children):
        for c in children:
            clickable = c['clickable']
            if(clickable):
                controlComponents.append((clickable, c['bounds'], 
                                          c['componentLabel']))
            elif("children" in c):
                loopForControlComponents(c["children"])
                
    loopForControlComponents(children)
    ccCount = len(controlComponents)
    return controlComponents, ccCount

## Define Matrix Structure
The matrix is defined as such that any connection flowing downwards from a node above is marked in the matrix row that defines the node existing underneat it with a value 1. In other words if the first node flows to the leftmost node below it from the left we have a matrix:<br/>
[[0,0,0],<br/>
 [1,0,0],<br/>
 [0,0,0]]<br/>
 In the above definition one node is not connected to anything.
 If node on second level had a connection to two other nodes underneath it we would define the matrix as:<br/>
 [[0,0,0,0],<br/>
  [1,0,0,0],<br/>
  [0,1,0,0],<br/>
  [0,1,0,0]]<br/>

In [4]:
def defineAdjacencyMatrix(controlComponents, ccCount): 
    adjacencyMatrix =  [[0] * (ccCount+2)]
    endVector = [0] * (ccCount+2) 
    endpointsCounter = ccCount
    #this defines where the next nodes shall form connections
    currentAnchorColumn = 0
    componentLimit = ccCount - 1
    for i in range(ccCount): 
            component = controlComponents[i]
            intermediateVector = []
            
            #special case for filling a login code with automatic UI refresh 
            #when Input field meets required length
            if(component[2] == 'Input' and i == componentLimit):
                intermediateVector = [0] * (ccCount+1)
                intermediateVector.insert(currentAnchorColumn, 1)
                currentAnchorColumn = 0
                endVector[i+1] = 1
            elif(component[2] == 'Input'):
                intermediateVector = [0] * (ccCount+1) 
                intermediateVector.insert(currentAnchorColumn, 1) 
                currentAnchorColumn = i + 1
                endVector[i+1] = 0
                endpointsCounter -= 1
            else:
                intermediateVector = [0] * (ccCount+1)
                intermediateVector.insert(currentAnchorColumn, 1)
                currentAnchorColumn = 0
                endVector[i+1] = 1
            adjacencyMatrix = adjacencyMatrix + [intermediateVector] 
            #endVector[id+1] = 1
    adjacencyMatrix = adjacencyMatrix + [endVector] 
    print("adjacencymatrix")
    for a in adjacencyMatrix:
        print(a)
    return np.array(adjacencyMatrix), endpointsCounter


def defineVertices(adjacencyMatrix, controlComponents): 
    verticeLabels = ["E"] 
    
    noEndpoints = pd.DataFrame(adjacencyMatrix)
    noEndpoints.drop(0,inplace=True)
    noEndpoints.drop(noEndpoints.tail(1).index,inplace=True)
    layersWithVertices = noEndpoints.sum(axis=0) 
    layersWithVertices = layersWithVertices[layersWithVertices != 0]
    columnsWithVerticesIndexes = layersWithVertices.index
    currentComponentItem = 0
    for i in columnsWithVerticesIndexes:
        currentColumn = adjacencyMatrix[:, i]
        for row, c in enumerate(currentColumn):
            if c == 1:
                currentItem = controlComponents[currentComponentItem][2]
                #These form the base level of the graph and other components are inserted around these
                #to match the order required by igraph
                if(i == 0):
                    verticeLabels.append(verticesMap[currentItem])
                else:
                    verticeLabels.insert(row-1, verticesMap[currentItem])
                currentComponentItem += 1

    #verticesPerLayer= layersWithVertices.iloc[layersWithVertices.to_numpy().nonzero()[0]].to_numpy()
    #for c in controlComponents: 
    #    if(c[2] in verticesMap): 
    #        print("c", c[2], "v", verticesMap[c[2]])
    #        verticeLabels = verticeLabels + [verticesMap[c[2]]]
    #verticeLabels = ["E"] + verticeLabels 
    verticeLabels = verticeLabels + ["S"] 
    return verticeLabels 

def defineEdges(adjacencyMatrix, verticeLabels, endpointCount):
    edgeLabels = []

    #edgesPerLayer.loc[:, (edgesPerLayer != 0).any(axis=0)]
    #print("--")
    #print(noEndpoints)
    #print("--")
    #print(edgesPerLayer)
    
    #currentEdgeGroup = 0
    workedThroughGroupItems = 0
    for v in range(len(verticeLabels)-1):
        if(verticeLabels[v] in edgesMap):
            
            edgeLabels = edgeLabels + [edgesMap[verticeLabels[v]]]
            #currentEdge = 0
        elif(verticeLabels[v] == "E"):
            edgeLabels = (["O"]*endpointCount) + edgeLabels
    #for v in range(len(verticeLabels)-1,-1,-1):
    #    print("V is", v, verticeLabels[v])
    #    if(verticeLabels[v] in edgesMap):
    #        edgeLabels = edgeLabels + [edgesMap[verticeLabels[v]]]
    #    elif(verticeLabels[v] == "E"):
    #        edgeLabels = (["O"]*endpointCount) + edgeLabels
    return edgeLabels

## Complexity Computations

In [5]:
def computeHalstead(edgeLabels, verticeLabels):
    #number of unique operators
    n1 = len(set(edgeLabels))
    #number of unique operands. We remove start and end
    n2 = len(set(verticeLabels))-2
    #total frequency of operators
    N1 = len(edgeLabels)
    #total frequency of operands. We remote start and end 
    N2 = len(verticeLabels)-2
    
    firstPart = (n1*N2*(N1+N2))/(2*n2)
    logPart = math.log(n1+n2, 2)
    #rounded to S significant numbers
    return round(firstPart*logPart,3)


def computeMcCabeVG(adjacencyMatrix):
    nodes = len(adjacencyMatrix)
    #vertices = 0
    edges = adjacencyMatrix.sum().sum()
   # for a in adjacencyMatrix:
    #    print(a)
    #    vertices += a.count(1)
    return nodes - edges

#called from computeWoodEntropy()
def woodComponentComplexity(adjacencyMatrix):
    print("xx")

def computeWoodEntropy(adjacencyMatrix):
    #print("type", type(adjacencyMatrix))
    print("")

## Call function and visualization

In [6]:
def getFile(fileName):
    data = json.load(open("hierarchies/"+fileName, encoding="utf-8"))
    dimensions = data['bounds']
    children = data['children']
    return dimensions, children

def visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, fileName, manualTest=False):
    directed = ig.Graph.Adjacency(adjacencyMatrix, mode="directed") 
    directed.vs["label"] = verticeLabels
    directed.es["label"] = edgeLabels
    layout = directed.layout("rt")
    #save to file
    ig.plot(directed, "graphs/"+fileName[:-5]+".png", layout=layout, bbox=(0, 0, 800, 800), margin=20)
    #return as a plot for a jupyter cell
    if(manualTest == True):
        return ig.plot(directed, layout=layout, bbox=(0, 0, 350, 350), margin=20) 

def determineControlGraphForFile(file):
    dimensions, children = getFile(file)
    controlComponents, ccCount = getControlComponents(children)
    #we need to sort control components as they are not always
    #in a sensible order from the perspective of how the actions would be performed
    sortedControlComponents = sortControlComponents(controlComponents)
    adjacencyMatrix, endpointCount = defineAdjacencyMatrix(sortedControlComponents, ccCount)
    verticeLabels = defineVertices(adjacencyMatrix, sortedControlComponents)
    edgeLabels = defineEdges(adjacencyMatrix, verticeLabels, endpointCount)

    #complexity computation
    halstead = computeHalstead(edgeLabels, verticeLabels)
    mcCabe = computeMcCabeVG(adjacencyMatrix)
    wood = computeWoodEntropy(adjacencyMatrix)
    print("Halstead", halstead, "McCabe", mcCabe, "Wood", wood)
    return adjacencyMatrix, verticeLabels, edgeLabels

## Generate graphs for all json files

In [7]:
def listAllFiles(folderPath):
    return os.listdir(folderPath)

def loopAllJsonFiles(folderPath):
    allFiles = listAllFiles(folderPath)
    for f in allFiles:
        print("------------")
        print("Starting file", f)
        print("------------")
        adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile(f)
        visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, f)

In [12]:
loopAllJsonFiles("hierarchies")

------------
Starting file 10128.json
------------
adjacencymatrix
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 0]

Halstead 19.02 McCabe 0 Wood None
------------
Starting file 10180.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 0]

Halstead 58.048 McCabe -3 Wood None
------------
Starting file 10183.json
------------
adjacencymatrix
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 0]

Halstead 12.0 McCabe 0 Wood None
------------
Starting file 10208.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 0]

Halstead 83.589 McCabe -4 Wood None
------------
Starting file 10214.json
------------
adjacencymatrix
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 0]

Halstead 19

adjacencymatrix
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 0]

Halstead 48.0 McCabe -2 Wood None
------------
Starting file 11177.json
------------
adjacencymatrix
[0, 0, 0]
[1, 0, 0]
[0, 1, 0]

Halstead 4.755 McCabe 1 Wood None
------------
Starting file 11231.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 0]

Halstead 75.0 McCabe -3 Wood None
------------
Starting file 11233.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 0]

Halstead 37.151 McCabe -2 Wood None
------------
Starting file 11235.json
------------
adjacencymatrix
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 1, 1, 1, 0]

Halstead 27.0 McCabe -1 Wood None
------------
Starting file 11347.json
---------

adjacencymatrix
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 0]

Halstead 12.0 McCabe 0 Wood None
------------
Starting file 12675.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 1, 1, 0]

Halstead 167.179 McCabe -2 Wood None
------------
Starting file 12775.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

adjacencymatrix
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 1, 1, 0]

Halstead 41.795 McCabe 0 Wood None
------------
Starting file 14073.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 1, 1, 0]

Halstead 167.179 McCabe -2 Wood None
------------
Starting file 14085.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 1, 1, 0]

Halstead 167.179 McCabe -2 Wood None
------------
Starting file 14143.json
------------
adjacencymatrix
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 1, 1, 1, 0]

Halstead 42.794 McCabe -1 Wood None
------------
Starting file 14247.json
------------
adjacencymatrix
[0, 0, 

adjacencymatrix
[0, 0, 0]
[1, 0, 0]
[0, 1, 0]

Halstead 4.755 McCabe 1 Wood None
------------
Starting file 15449.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 0]

Halstead 69.794 McCabe -4 Wood None
------------
Starting file 15452.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 0]

Halstead 58.048 McCabe -3 Wood None
------------
Starting file 15574.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 0]

Halstead 58.048 McCabe -3 Wood None
------------
Starting file 15631.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[

...

Starting file 30450.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 0]

Halstead 48.0 McCabe -2 Wood None
------------
Starting file 30518.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 0]

Halstead 113.774 McCabe -5 Wood None
------------
Starting file 30542.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 

adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 0]

Halstead 192.0 McCabe -6 Wood None
------------
Starting file 32115.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 0]

Halstead 58.048 McCabe -3 Wood None
------------
Starting file 32214.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 

adjacencymatrix
[0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 1, 0]

Halstead 113.194 McCabe -1 Wood None
------------
Starting file 33303.json
------------
adjacencymatrix
[0, 0, 0]
[1, 0, 0]
[0, 1, 0]

Halstead 4.755 McCabe 1 Wood None
------------
Starting file 33393.json
------------
adjacencymatrix
[0, 0, 0]
[1, 0, 0]
[0, 1, 0]

Halstead 4.755 McCabe 1 Wood None
------------
Starting file 33395.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 0]

Halstead 147.0 McCabe -5 Wood None
------------
Starting file 33407.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 

adjacencymatrix
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 0]

Halstead 19.02 McCabe 0 Wood None
------------
Starting file 34509.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 0]

Halstead 83.589 McCabe -4 Wood None
------------
Starting file 34521.json
------------
adjacencymatrix
[0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 1, 1, 1, 1, 0]

Halstead 37.151 McCabe -2 Wood None
------------
Starting file 34535.json
------------
adjacencymatrix
[0, 0, 0]
[1, 0, 0]
[0, 1, 0]

Halstead 4.755 McCabe 1 Wood None
------------
Starting file 3458.json
------------
adjacencymatrix
[0, 0, 0]
[1, 0, 0]
[0, 1, 0]

Halstead 4.755 McCabe 1 Wood None
------------
Starting file 34595.json
------------
adjacencymatrix
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 

KeyError: 'Drawer'

# Manual Test Section To Improve Code

### ID 23689

In [ ]:
fileName = "23689.json"
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile(fileName)
print(verticeLabels)
print(edgeLabels)
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, fileName, True)

### ID 21545

In [ ]:
fileName = "21545.json"
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile(fileName)
print(verticeLabels)
print(edgeLabels)
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, fileName, True)

### ID 20891 

In [ ]:
fileName = "20891.json"
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile(fileName)
print(verticeLabels)
print(edgeLabels)
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, fileName, True)

### Id 1996
Outcome: deleted due to faulty json

In [ ]:
fileName = "1996.json"
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile(fileName)
print(verticeLabels)
print(edgeLabels)
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, fileName, True)

### ID 17467

In [ ]:
fileName = "17467.json"
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile(fileName)
print(verticeLabels)
print(edgeLabels)
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, fileName, True)

### ID 14310

In [ ]:
fileName = "14310.json"
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile(fileName)
print(verticeLabels)
print(edgeLabels)
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, fileName, True)

### ID 10594

In [ ]:
fileName = "10594.json"
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile(fileName)
print(verticeLabels)
print(edgeLabels)
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, fileName, True)

### ID 10482 computation

In [ ]:
fileName = "10482.json"
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile(fileName)
print(verticeLabels)
print(edgeLabels)
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, fileName, True)

### ID 1809 computation

In [ ]:
fileName = "1809.json"
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile(fileName)
print(verticeLabels)
print(edgeLabels)
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, fileName, True)

## Id 492 computation

In [ ]:
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile("492.json")
print(verticeLabels)
print(edgeLabels)
#verticeLabels = ['E', 'I', 'I', 'TB', 'I', 'TB', 'IP', 'S']
edgeLabels = ['O', 'O', 'O', 'O', '0', 'Tap', 'Tap', 'Tap', 'Tap', 'Tap', 'Input']
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, True)

### Id 1863 computation

In [ ]:
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile("1863.json")
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, True)

### Id 546 computation

In [ ]:
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile("546.json")
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, True)

### Id 758 computation

In [ ]:
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile("758.json")
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, True)

### Id 10596 computation

In [ ]:
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile("10596.json")
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, True)

### Id 1074 Computation

In [ ]:
adjacencyMatrix, verticeLabels, edgeLabels = determineControlGraphForFile("1074.json")
visualizeMatrix(adjacencyMatrix, verticeLabels, edgeLabels, True)